In [19]:
from whoosh.index import open_dir
from sentence_transformers import SentenceTransformer
from whoosh.qparser import QueryParser
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModel,AutoModelForTokenClassification,AutoModelForSequenceClassification
import torch
from sklearn.metrics.pairwise import cosine_similarity
import torch.nn.functional as F
import nltk
#nltk.download("punkt")
from keybert import KeyBERT
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from transformers import TrainingArguments, AdamW
import torch.nn as nn
from transformers import BertTokenizer
from sklearn.decomposition import PCA
import time

kw_model = KeyBERT()




def sustract_keywords(query):
    resul=[]
    kw = kw_model.extract_keywords(query,keyphrase_ngram_range=(1, 1), stop_words=None)
    for k,v in kw:
        resul.append(k)
        
    return resul

def build_query(lista):
    new_l=[]
    for elem in lista:
        
        new_l.append(f'evidences:{elem} OR keywords:{elem}')
        
    return " OR ".join(new_l)

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


modelSBERT = SentenceTransformer('mitra-mir/setfit-model-Feb11-Misinformation-on-Media-Traditional-Social')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Pepe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Modelo para sacar las 5 evidencias más cercanas a cada claim del dataset y sacar, obviamente, los embeddings

data=pd.read_csv('iverifyzm.csv')
data=data.drop(['Unnamed: 0'], axis=1)
for i in range(len(data)):
    if data['label'][i]=='True' :
        data['label'][i]=2
    elif data['label'][i]=='Partly False' or data['label'][i]=='False' or data['label'][i]=='Misleading':
        data['label'][i]=0
    elif data['label'][i]=='Unproven':
        data['label'][i]=1
        
d=pd.read_csv('africa_check_nigeria.csv')
d = d.rename(columns={'claim':'claims'})
for i in range(len(d)):
    if d['label'][i]=='checked' or d['label'][i]=='Checked':
        d['label'][i]=2
    elif d['label'][i]=='false' or d['label'][i]=='False' or d['label'][i]=='Fake' or d['label'][i]=='Misleading':
        d['label'][i]=0
        
data=pd.concat([data,d])
data

cont = 1
embedding_train = []
evidence_sentences = []
for ev in data['evidences']:
    print(f"Analizando {cont}")
    cont += 1
    for evs in ev.split('.'):
        if len(evs) > 10:
            evidence_sentences.append(evs.strip())
            embedding_train.append(modelSBERT.encode(evs.strip()))

cont = 1
top5 = []
for claim in data['claims']:
    print(f"Analizando {cont}")
    cont += 1
    emb_claim = modelSBERT.encode(claim.strip())
    indx = np.flip(np.argsort(cosine_similarity(emb_claim.reshape(1, -1), embedding_train))[0][-5:])
    claim5 = []
    for i in indx:
        claim5.append(evidence_sentences[i])
    top5.append(claim5)
    
newevidences = []
for t in top5:
    newevidences.append('. '.join(t))
newevidences

train = pd.DataFrame(zip(data['claims'], newevidences, data['label']), columns = ['claim', 'top5evidences', 'label'])
train

In [8]:
#train=pd.read_csv('train.csv')

In [27]:
train=pd.read_csv('elections_fact_check_extrafeatures.csv')

In [28]:
t=0

f=0

for i in range(len(train)):
    if train['label'][i]==0:
        f+=1
    elif train['label'][i]==1:
        t+=1
    
        
print(f'True: {t}')
print(f'False: {f}')


True: 2887
False: 3297


train.to_csv('train.csv', header=True, index=False)

### Otros modelos y tokenizers

#tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/stsb-roberta-large')
"""model = AutoModelForSequenceClassification.from_pretrained("roberta-large-mnli", num_labels=3,
                                                          ignore_mismatched_sizes=True)"""
#tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

In [92]:

tokenizer = AutoTokenizer.from_pretrained("roberta-large-mnli")
tokenized_dataset = []
for i,row in train[0:1000].iterrows():
    tokenized_dataset.append(tokenizer(row["claim"], row['top5_evidences'],
                             pad_to_max_length = True,  max_length = 256,return_attention_mask=True,add_special_tokens = True
                            ,return_tensors = 'pt'))
    
    



model = AutoModelForSequenceClassification.from_pretrained("vikram71198/distilroberta-base-finetuned-fake-news-detection", num_labels=3,
                                                          ignore_mismatched_sizes=True)





Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vikram71198/distilroberta-base-finetuned-fake-news-detection and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [94]:
X_train, X_val,  y_train, y_val = train_test_split(tokenized_dataset,train['label'][0:1000], test_size = 0.2)

train_input_ids = torch.cat([d['input_ids'] for d in X_train],dim=0)
train_attention_masks = torch.cat([d['attention_mask'] for d in X_train],dim=0)
#train_labels = torch.tensor(y_train.map({0: 0,  1: 1,  2: 2}).values)
train_labels = torch.tensor(y_train.map({0: 0,  1: 1}).values)
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
#train_dataset = TensorDataset(train_input_ids,  train_labels)

val_input_ids = torch.cat([d['input_ids'] for d in X_val],dim=0)
val_attention_masks = torch.cat([d['attention_mask'] for d in X_val],dim=0)
val_labels = torch.tensor(y_val.map({0: 0,  1: 1}).values)
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)
#val_dataset = TensorDataset(val_input_ids, val_labels)

batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Crear un DataLoader que divide el conjunto de datos de val en lotes y les aplica aleatorización y paralelización en la CPU o GPU
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [95]:
len(train_dataset)

800

### Congelamos todas las capas menos las de clasificación, las 5 últimas

In [83]:
"""count=0
for name, param in model.named_parameters():
    count+=1
    if count <390:
        param.requires_grad = False"""
    

In [99]:
cont=0
for name, param in model.named_parameters():
    print(name)
    if count <101:
        param.requires_grad = False

roberta.embeddings.word_embeddings.weight
roberta.embeddings.position_embeddings.weight
roberta.embeddings.token_type_embeddings.weight
roberta.embeddings.LayerNorm.weight
roberta.embeddings.LayerNorm.bias
roberta.encoder.layer.0.attention.self.query.weight
roberta.encoder.layer.0.attention.self.query.bias
roberta.encoder.layer.0.attention.self.key.weight
roberta.encoder.layer.0.attention.self.key.bias
roberta.encoder.layer.0.attention.self.value.weight
roberta.encoder.layer.0.attention.self.value.bias
roberta.encoder.layer.0.attention.output.dense.weight
roberta.encoder.layer.0.attention.output.dense.bias
roberta.encoder.layer.0.attention.output.LayerNorm.weight
roberta.encoder.layer.0.attention.output.LayerNorm.bias
roberta.encoder.layer.0.intermediate.dense.weight
roberta.encoder.layer.0.intermediate.dense.bias
roberta.encoder.layer.0.output.dense.weight
roberta.encoder.layer.0.output.dense.bias
roberta.encoder.layer.0.output.LayerNorm.weight
roberta.encoder.layer.0.output.LayerNorm

In [100]:
optim = AdamW(model.parameters(), lr=1e-4)
loss_fn = nn.CrossEntropyLoss()
# Número de épocas que deseas entrenar
num_epochs = 10

# Entrenamiento
for epoch in range(num_epochs):
    # Entrenamiento del modelo
    model.train()
    for batch_idx, (inputs, masks, target) in enumerate(train_dataloader):
        # Reinicia el gradiente acumulado en el optimizador
        optim.zero_grad()

        # Pasa los datos por el modelo para obtener las predicciones
        
        output = model(input_ids=inputs, attention_mask=masks, labels=target)
        #print(output)
        # Calcula la pérdida
        loss = loss_fn(output.logits, target)

        # Retropropagación
        loss.backward()

        # Actualiza los parámetros del modelo
        optim.step()

    # Validación del modelo
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for (inputs, masks, target) in val_dataloader:
            # Pasa los datos por el modelo para obtener las predicciones
            output = model(input_ids=inputs, attention_mask=masks, labels=target)

            # Predice la clase con la mayor probabilidad
            _, predicted = torch.max(output.logits, 1)

            # Calcula el número de predicciones correctas y el número total de ejemplos
            total += target.size(0)
            correct += (predicted == target).sum().item()

        # Calcula la precisión en la validación
        val_acc = correct / total

    # Imprime la precisión en la validación
    print('Epoch: {}, Val Acc: {}'.format(epoch, val_acc))

Epoch: 0, Val Acc: 0.88
Epoch: 1, Val Acc: 0.89
Epoch: 2, Val Acc: 0.89


KeyboardInterrupt: 

### Creación del dataset de embedding (claim+top5evidences) + label + sentiment analysis

In [33]:
strain=train[0:1000]
t=0

f=0

for i in range(len(strain)):
    if strain['label'][i]==0:
        f+=1
    elif strain['label'][i]==1:
        t+=1
    
        
print(f'True: {t}')
print(f'False: {f}')


True: 492
False: 508


### Agrupamos los embeddings utilizando PCA para reducir su dimensión

In [34]:


strain['evclaim']=strain['claim']+strain['top5_evidences']
ev=strain['evclaim']
emb=[]
cont=0
for e in ev:
    cont+=1
    print(f'Analizando {cont}')
    
    emb.append(modelSBERT.encode(e.strip()))
    
pca_model=PCA(n_components=50)
pca_model.fit(emb)
print("Sum of variance ratios: ",sum(pca_model.explained_variance_ratio_))



C:\Users\Pepe\AppData\Local\Temp\ipykernel_15964\2165557184.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  strain['evclaim']=strain['claim']+strain['top5_evidences']


Analizando 1
Analizando 2
Analizando 3
Analizando 4
Analizando 5
Analizando 6
Analizando 7
Analizando 8
Analizando 9
Analizando 10
Analizando 11
Analizando 12
Analizando 13
Analizando 14
Analizando 15
Analizando 16
Analizando 17
Analizando 18
Analizando 19
Analizando 20
Analizando 21
Analizando 22
Analizando 23
Analizando 24
Analizando 25
Analizando 26
Analizando 27
Analizando 28
Analizando 29
Analizando 30
Analizando 31
Analizando 32
Analizando 33
Analizando 34
Analizando 35
Analizando 36
Analizando 37
Analizando 38
Analizando 39
Analizando 40
Analizando 41
Analizando 42
Analizando 43
Analizando 44
Analizando 45
Analizando 46
Analizando 47
Analizando 48
Analizando 49
Analizando 50
Analizando 51
Analizando 52
Analizando 53
Analizando 54
Analizando 55
Analizando 56
Analizando 57
Analizando 58
Analizando 59
Analizando 60
Analizando 61
Analizando 62
Analizando 63
Analizando 64
Analizando 65
Analizando 66
Analizando 67
Analizando 68
Analizando 69
Analizando 70
Analizando 71
Analizando 72
A

Analizando 555
Analizando 556
Analizando 557
Analizando 558
Analizando 559
Analizando 560
Analizando 561
Analizando 562
Analizando 563
Analizando 564
Analizando 565
Analizando 566
Analizando 567
Analizando 568
Analizando 569
Analizando 570
Analizando 571
Analizando 572
Analizando 573
Analizando 574
Analizando 575
Analizando 576
Analizando 577
Analizando 578
Analizando 579
Analizando 580
Analizando 581
Analizando 582
Analizando 583
Analizando 584
Analizando 585
Analizando 586
Analizando 587
Analizando 588
Analizando 589
Analizando 590
Analizando 591
Analizando 592
Analizando 593
Analizando 594
Analizando 595
Analizando 596
Analizando 597
Analizando 598
Analizando 599
Analizando 600
Analizando 601
Analizando 602
Analizando 603
Analizando 604
Analizando 605
Analizando 606
Analizando 607
Analizando 608
Analizando 609
Analizando 610
Analizando 611
Analizando 612
Analizando 613
Analizando 614
Analizando 615
Analizando 616
Analizando 617
Analizando 618
Analizando 619
Analizando 620
Analizando

NameError: name 'emb_comps' is not defined

In [62]:
emb_comps = pca_model.transform(emb)
emb_comps.shape

emb_comps=list(emb_comps)
train_ev_label=pd.DataFrame()
train_ev_label['ev']=emb_comps
train_ev_label['label']=train['label']

In [65]:
sent=train[list(train.columns)[4:]][0:1000]

In [66]:
train_ev_label=pd.concat([train_ev_label,sent],axis=1)

In [67]:
train_ev_label

,ev,label,probFalseZS,probTrueZS,emotion_max,category_max,admiration,amusement,anger,annoyance,...,severe_toxicity,sexual_explicit,threat,toxicity,white,Campaign strategies,Election administration,Voter registration,Voter turnout,Women in politics
0,"[0.10537340865815431, -0.23315448790546983, 0....",0,0.972272,0.027728,neutral,Election administration,0.000132,0.000190,0.000024,0.000100,...,8.902355e-07,0.000014,0.000022,0.000433,0.000061,0.207005,0.574061,0.053560,0.106275,0.059098
1,"[-0.22703017801758402, -0.13707236571822345, 0...",1,0.315393,0.684607,neutral,Campaign strategies,0.000294,0.000132,0.000056,0.000283,...,1.159646e-06,0.000018,0.000023,0.000350,0.000076,0.504780,0.120901,0.056547,0.190566,0.127206
2,"[0.43777697114019143, -0.19565933482684034, 0....",0,0.732823,0.267177,disapproval,Campaign strategies,0.000228,0.000323,0.001817,0.009162,...,8.240283e-06,0.015960,0.000170,0.011057,0.000073,0.379447,0.113501,0.084640,0.228957,0.193455
3,"[-0.3795883842895938, -0.13462324753925564, 0....",0,0.987513,0.012487,approval,Campaign strategies,0.000186,0.000165,0.000047,0.000338,...,1.097509e-06,0.000017,0.000023,0.000346,0.000085,0.427851,0.078924,0.086639,0.176794,0.229791
4,"[0.745929506255487, 0.17149958777612329, -0.04...",0,0.573740,0.426261,neutral,Campaign strategies,0.000569,0.000149,0.000014,0.000071,...,1.063706e-06,0.000017,0.000021,0.000380,0.000088,0.413757,0.124321,0.050157,0.234383,0.177382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"[0.719550687332116, 0.15117377243908478, -0.10...",1,0.786035,0.213964,neutral,Campaign strategies,0.000589,0.000364,0.000074,0.000554,...,1.060246e-06,0.000019,0.000027,0.000478,0.000069,0.454844,0.129144,0.095846,0.147722,0.172444
996,"[-0.42748668246252075, -0.12071436330357133, -...",1,0.935927,0.064073,disapproval,Campaign strategies,0.000047,0.000088,0.000244,0.001209,...,1.079972e-06,0.000019,0.000028,0.000430,0.000062,0.660216,0.105366,0.080979,0.085341,0.068099
997,"[0.7364865079224981, 0.05463787486491473, -0.0...",1,0.352699,0.647301,neutral,Campaign strategies,0.000318,0.000388,0.000048,0.000473,...,1.053700e-06,0.000020,0.000025,0.000426,0.000068,0.657777,0.099215,0.068443,0.065280,0.109284
998,"[0.7733090724259504, 0.1652767634955828, -0.11...",1,0.962491,0.037509,neutral,Campaign strategies,0.000215,0.000085,0.000466,0.000951,...,9.267620e-07,0.000018,0.000035,0.000929,0.000068,0.457006,0.185236,0.044701,0.101398,0.211659


### Nos guardamos el nuevo dataset que se llama train_ev_label

In [101]:
#train_ev_label.to_csv('emblabelssentiments.csv', header=True, index=False)
#train_ev_label=pd.read_csv('emblabelssentiments.csv')

### Probamos con este nuevo dataset a realizar pruebas con Random Forest o SVM

In [69]:
x_train,x_test,y_train,y_test = train_test_split(emb_comps,train_ev_label['label'],test_size=0.2,random_state=42)

In [70]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(x_train,y_train)
print("Score of RFC",rfc.score(x_test,y_test))

Score of RFC 0.815


In [71]:
from sklearn.svm import SVC
start = time.time() 

svm_classifier = SVC()
svm_classifier.fit(x_train,y_train)

end = time.time()
process = round(end-start,2)
print("Support Vector Machine Classifier has fitted, this process took {} seconds".format(process))

Support Vector Machine Classifier has fitted, this process took 0.03 seconds


In [72]:
svm_classifier.score(x_test,y_test)

0.84

In [74]:
test=svm_classifier.predict(x_test)
test

array([0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 0], dtype=int64)

In [75]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test,y_test)

array([[87, 22],
       [10, 81]], dtype=int64)